# Decoding with Belief Propagation - Working to take some for loops out of the game

We want to iterate the following the following equations:

$$ \hat{m}_{\mu j} = \tanh ( \beta(\rho) J_\mu ) \prod_{ l \in {\cal L} (\mu) \backslash j } m_{\mu l}  \; \; ,  $$ 

$$ m_{\mu j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) \backslash \mu } \ \tanh^{-1} \left(  \hat{m}_{\nu j}  \right)    + \beta(\rho_\xi)  \right)   \; \; ,  $$  

which are Eqs.(96) from the paper [Low-density parity-check codes—A statistical physics perspective](https://www.sciencedirect.com/science/article/pii/S1076567002800180), by R. Vicente, D. Saad and Y. Kabashima.

The function $ \beta(x) $ is the Nishimori temperature,

$$ \beta(x) = \frac{1}{2} \log \left(  \frac{1- \rho}{\rho}  \right) \; \; .  $$

The quantity $\rho$ is the flip probablility of the noisy channel (BSC),

$$ P ( J | J^{(0)} ) = (1 - \rho) \delta_{J, J^{(0)} } + \rho \delta_{J, -J^{(0)} }   \; \; , $$

whereas the prior distribution for each meassage bit is assumed to be

$$ P ( S_j ) = (1 - \rho_\xi) \delta_{+1, S_j }  +  \rho_\xi \delta_{-1, S_j }  \; \; .  $$

The object $ {\cal L} (\mu)$ represents the set of $K$ non-zero elements on the row $\mu$ of the code generator matrix ${\cal G}$ (the one which adds redundancy), 

$$  {\cal L} (\mu) = \langle i_1, i_2, ..., i_K \rangle  \; \; .  $$

The are $C$ non-zero elements per column on the matrix ${\cal G}$:

$$ \sum_{\mu : j \in {\cal L}(\mu)} i_j = C \; \; ; \; \; \forall j = 1, ..., K \; \; . $$

The object $ {\cal M} (j)$ represents the set of all index sets that contain $j$.

After convergence of the iterative procedure, we can calculate the pseudo-posterior:

$$ m_{j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) } \tanh^{-1} \left(  \hat{m}_{\nu j}  \right)    + \beta(\rho_\xi)  \right)   \; \; ,  $$
from which the Bayes optimal estimate is obtained
$$ \hat{\xi}_j  = sign( m_j ) $$ 

In [18]:
import torch
import numpy as np
import matplotlib.pyplot as plt

Defining the variables of the problem:

In [19]:
# Message lengh
N = 10

# Codeword lengh
M = 20

# Non-zero elements per row of the generation matrix
K = 4

# Number of messages
n = 15

# Noisy channel
p = 0.5
beta = 0.5*np.log( (1 - p) / p)

# Message prior
p_prior = 0.1
beta_prior = 0.5*np.log( (1 - p_prior) / p_prior)

## Generating messages

Each message is a $N$ dimensional vector. Generate a set of $n$ messages.

In [24]:
random = torch.rand([n, N])

#### -1 with probability p_prior
#### +1 with probability 1 - p_prior

message = 2* (random > p_prior).float() - 1

In [25]:
print(message)
message.shape

tensor([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1., -1.],
        [ 1.,  1.,  1., -1.,  1.,  1., -1.,  1.,  1.,  1.],
        [ 1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,  1.]])


torch.Size([15, 10])

## Encoding

Each message is encoded to a high dimensional vector ${\bf J}^{(0)} \in \{ \pm 1  \}^M$ defined as 

$$   J^{(0)}_{\langle i_1, i_2, ...., i_K \rangle} = \xi_1 \xi_ 2 ... \xi_K  \; \; ,$$

where $M$ sets of $K \in [ 1, ..., N]$ indexes are randomly chosen.

In [26]:
encoding = torch.randint(0, N, [M, K])

In [27]:
encoding.shape

torch.Size([20, 4])

From `encoding`, we construct the encoded message ${\bf J}^{(0)}$.

In [28]:
# Initializing
J0 = torch.take(message[0], encoding).prod(dim=1)
J0 = J0.unsqueeze(0)

# Loop over all messages
for j in range(1, message.shape[0]):
    
    J0_ = torch.take(message[j], encoding).prod(dim=1)
    J0_ = J0_.unsqueeze(0)
    
    J0 = torch.cat((J0, J0_), dim= 0)

In [64]:
print(J0)
J0.shape

tensor([[ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1., -1.,  1., -1., -1., -1.,  1.,  1., -1.,  1.,  1., -1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.],
        [ 1., -1., -1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1.,  1.,
         -1., -1.,  1., -1., -1.,  1.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
          1.,  1.,  1.,  1.,  1.,  1.],
        [-1., -1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1., -1., -1., -1.,
         -1.,  1.,  1.,  1., -1.,  1.],
        [-1., -1.,  1.,  1.,  1., -1., -1., -1.,  1., -1., -1.,  1., -1., -1.,
          1.,  1., -1., -1.,  1., -1.],
        [ 1.,  1.,  1., -1.,  1.,  1.,  1.,  1.,

torch.Size([15, 20])

## Corrupted version of the encoded set of messages

In [77]:
random = torch.rand(J.shape)

## Flip tensor: -1 with probability p
flip = 2*(random <= p).float() - 1

## J corrupted version: element wise multiplication
J = torch.mul(J0, flip)

In [80]:
print(J)
J.shape

tensor([[ 1.,  1., -1.,  1.,  1., -1.,  1.,  1., -1., -1., -1., -1., -1., -1.,
          1., -1., -1.,  1., -1., -1.],
        [ 1.,  1., -1., -1.,  1., -1.,  1., -1.,  1.,  1.,  1.,  1., -1.,  1.,
         -1.,  1.,  1., -1.,  1.,  1.],
        [-1., -1.,  1.,  1.,  1.,  1.,  1.,  1., -1.,  1., -1.,  1., -1., -1.,
         -1., -1., -1., -1., -1., -1.],
        [-1., -1.,  1., -1.,  1., -1.,  1.,  1., -1., -1., -1.,  1.,  1., -1.,
          1., -1., -1., -1.,  1.,  1.],
        [-1.,  1.,  1., -1.,  1.,  1.,  1., -1.,  1., -1.,  1.,  1.,  1., -1.,
          1.,  1.,  1., -1., -1.,  1.],
        [-1., -1., -1.,  1., -1.,  1., -1.,  1., -1.,  1., -1.,  1.,  1., -1.,
         -1.,  1., -1.,  1., -1.,  1.],
        [ 1., -1., -1., -1., -1., -1.,  1.,  1.,  1., -1., -1.,  1., -1., -1.,
          1., -1., -1.,  1., -1.,  1.],
        [-1., -1.,  1.,  1.,  1., -1., -1., -1.,  1.,  1., -1.,  1.,  1., -1.,
         -1., -1., -1.,  1.,  1., -1.],
        [-1.,  1., -1.,  1., -1.,  1.,  1.,  1.,

torch.Size([15, 20])

## Iterating BP equations for one message

Let us focus in one received message to iterate the belief propagation equations.

$$ \hat{m}_{\mu j} = \tanh ( \beta(\rho) J_\mu ) \prod_{ l \in {\cal L} (\mu) \backslash j } m_{\mu l}  \; \; ,  $$ 

$$ m_{\mu j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) \backslash \mu} \ \tanh^{-1} \left(  \hat{m}_{\nu j}  \right) + \beta(\rho_\xi)    \right)   \; \; ,  $$  

with $j = 1, ..., N$ and $\mu = 1, ..., M$.

We cal this message `J_`. We will worry later about a loop over all the received messages.

In [81]:
J_ = J[0]
print(J_)
print(J_.shape)

tensor([ 1.,  1., -1.,  1.,  1., -1.,  1.,  1., -1., -1., -1., -1., -1., -1.,
         1., -1., -1.,  1., -1., -1.])
torch.Size([20])


Random initialization of the beliefs $m_{\mu l}$.

In [82]:
m = torch.rand(M, N)

In [83]:
m

tensor([[0.8490, 0.7651, 0.4813, 0.4422, 0.4468, 0.5893, 0.6805, 0.3218, 0.1575,
         0.6146],
        [0.5140, 0.8164, 0.9899, 0.2906, 0.3551, 0.5489, 0.0058, 0.7731, 0.5287,
         0.6275],
        [0.7086, 0.9687, 0.7248, 0.6376, 0.1314, 0.1247, 0.0086, 0.3216, 0.9359,
         0.3480],
        [0.2044, 0.4443, 0.5222, 0.2857, 0.8737, 0.5808, 0.0553, 0.9413, 0.4699,
         0.3736],
        [0.9157, 0.9591, 0.3256, 0.3243, 0.8502, 0.6642, 0.4659, 0.5185, 0.3577,
         0.3249],
        [0.4693, 0.4397, 0.9608, 0.1113, 0.8198, 0.4708, 0.5807, 0.9098, 0.5703,
         0.3885],
        [0.4464, 0.8441, 0.5228, 0.5068, 0.4439, 0.5191, 0.4601, 0.9782, 0.0131,
         0.5136],
        [0.6102, 0.6242, 0.2763, 0.1424, 0.3031, 0.1346, 0.1362, 0.0520, 0.7587,
         0.5749],
        [0.0355, 0.0157, 0.3879, 0.9603, 0.2527, 0.3132, 0.2789, 0.3422, 0.3664,
         0.8268],
        [0.5327, 0.0029, 0.2150, 0.6878, 0.3713, 0.2700, 0.1526, 0.3893, 0.2898,
         0.9070],
        [0

Initialize an empty tensor to represent $\hat{m}_{\mu l}$.

In [84]:
m_hat = torch.empty(M, N)

In [85]:
m_hat

tensor([[ 6.2205e-20,  4.5792e-41,  6.2205e-20,  4.5792e-41, -7.0023e+02,
          3.0927e-41, -7.0023e+02,  3.0927e-41,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,         nan,         nan,  0.0000e+00,
          0.0000e+00,  1.4013e-45,  7.7312e-01,  0.0000e+00,  0.0000e+00],
        [ 1.2612e-44,  0.0000e+00, -2.1237e+26,  4.5790e-41, -1.1084e+02,
          3.0927e-41,         nan,         nan,  0.0000e+00,  0.0000e+00],
        [        nan,         nan, -2.1167e+26,  4.5790e-41,  1.4013e-45,
          5.8076e-01,  1.0089e-43,  0.0000e+00,  9.8091e-45,  0.0000e+00],
        [-2.1237e+26,  4.5790e-41, -1.1085e+02,  3.0927e-41,  0.0000e+00,
          0.0000e+00,  1.4013e-45,  0.0000e+00,  1.4013e-45,  0.0000e+00],
        [-2.1167e+26,  4.5790e-41,  1.4013e-45,  1.1133e-01,  2.0179e-43,
          0.0000e+00,  4.2039e-45,  0.0000e+00, -2.1237e+26,  4.5790e-41],
        [-1.1085e+02,  3.0927e-41,  0.0000e+00,  0.0000e+00,  1.4013e-45,
          0.0000e+00,  1.4013e-4

We want to calculate $\hat{m}_{\mu j}$.

$$ \hat{m}_{\mu j} = \tanh ( \beta(\rho) J_\mu ) \prod_{ l \in {\cal L} (\mu) \backslash j } m_{\mu l}  \; \; ,  $$ 

This first implementation has two `for` loops. This is potentially harmful if one cares about efficienty. We obviously do, but since we are just beginning, lets go on like this.

In [86]:
for mu in range(M):
    for j in range(N):
          
        # Keep only L(mu) which a are different of j
        index_no_j = torch.nonzero(encoding[mu] != j).squeeze()
        L_no_j = encoding[mu][index_no_j]
        
        # Message update        
        m_hat[mu, j] = torch.tanh( beta* J_[mu])*torch.take(m[mu], L_no_j).prod(dim=0)

In [89]:
J_.shape

torch.Size([20])

In [91]:
j = 10

In [98]:
encoding

tensor([[8, 8, 0, 6],
        [4, 5, 2, 6],
        [8, 8, 4, 8],
        [2, 6, 6, 1],
        [5, 1, 1, 8],
        [5, 2, 8, 3],
        [3, 9, 2, 8],
        [0, 2, 0, 3],
        [6, 9, 5, 6],
        [7, 2, 3, 8],
        [6, 9, 1, 5],
        [3, 6, 5, 4],
        [5, 7, 8, 6],
        [8, 6, 1, 8],
        [4, 2, 9, 1],
        [2, 1, 5, 4],
        [7, 3, 5, 2],
        [5, 3, 4, 0],
        [3, 7, 4, 6],
        [8, 8, 1, 3]])

In [113]:
encoding.shape

torch.Size([20, 4])

In [99]:
encoding != 8

tensor([[False, False,  True,  True],
        [ True,  True,  True,  True],
        [False, False,  True, False],
        [ True,  True,  True,  True],
        [ True,  True,  True, False],
        [ True,  True, False,  True],
        [ True,  True,  True, False],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True, False],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True, False,  True],
        [False,  True,  True, False],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [ True,  True,  True,  True],
        [False, False,  True,  True]])

In [114]:
torch.take(encoding, torch.nonzero(encoding != 8)).shape

torch.Size([66, 2])

In [109]:
encoding[encoding != 8]

tensor([0, 6, 4, 5, 2, 6, 4, 2, 6, 6, 1, 5, 1, 1, 5, 2, 3, 3, 9, 2, 0, 2, 0, 3,
        6, 9, 5, 6, 7, 2, 3, 6, 9, 1, 5, 3, 6, 5, 4, 5, 7, 6, 6, 1, 4, 2, 9, 1,
        2, 1, 5, 4, 7, 3, 5, 2, 5, 3, 4, 0, 3, 7, 4, 6, 1, 3])

In [104]:
print((encoding != 8).shape)
print(torch.nonzero(encoding != 8).shape)

torch.Size([20, 4])
torch.Size([66, 2])


In [105]:
torch.where(encoding != 8)

(tensor([ 0,  0,  1,  1,  1,  1,  2,  3,  3,  3,  3,  4,  4,  4,  5,  5,  5,  6,
          6,  6,  7,  7,  7,  7,  8,  8,  8,  8,  9,  9,  9, 10, 10, 10, 10, 11,
         11, 11, 11, 12, 12, 12, 13, 13, 14, 14, 14, 14, 15, 15, 15, 15, 16, 16,
         16, 16, 17, 17, 17, 17, 18, 18, 18, 18, 19, 19]),
 tensor([2, 3, 0, 1, 2, 3, 2, 0, 1, 2, 3, 0, 1, 2, 0, 1, 3, 0, 1, 2, 0, 1, 2, 3,
         0, 1, 2, 3, 0, 1, 2, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 3, 1, 2, 0, 1, 2, 3,
         0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 2, 3]))

In [115]:
torch.nonzero(encoding != 8, as_tuple= True)

(tensor([ 0,  0,  1,  1,  1,  1,  2,  3,  3,  3,  3,  4,  4,  4,  5,  5,  5,  6,
          6,  6,  7,  7,  7,  7,  8,  8,  8,  8,  9,  9,  9, 10, 10, 10, 10, 11,
         11, 11, 11, 12, 12, 12, 13, 13, 14, 14, 14, 14, 15, 15, 15, 15, 16, 16,
         16, 16, 17, 17, 17, 17, 18, 18, 18, 18, 19, 19]),
 tensor([2, 3, 0, 1, 2, 3, 2, 0, 1, 2, 3, 0, 1, 2, 0, 1, 3, 0, 1, 2, 0, 1, 2, 3,
         0, 1, 2, 3, 0, 1, 2, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 3, 1, 2, 0, 1, 2, 3,
         0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 0, 1, 2, 3, 2, 3]))

In [118]:
encoding[torch.nonzero(encoding != 8)]

tensor([[[8, 8, 0, 6],
         [8, 8, 4, 8]],

        [[8, 8, 0, 6],
         [2, 6, 6, 1]],

        [[4, 5, 2, 6],
         [8, 8, 0, 6]],

        [[4, 5, 2, 6],
         [4, 5, 2, 6]],

        [[4, 5, 2, 6],
         [8, 8, 4, 8]],

        [[4, 5, 2, 6],
         [2, 6, 6, 1]],

        [[8, 8, 4, 8],
         [8, 8, 4, 8]],

        [[2, 6, 6, 1],
         [8, 8, 0, 6]],

        [[2, 6, 6, 1],
         [4, 5, 2, 6]],

        [[2, 6, 6, 1],
         [8, 8, 4, 8]],

        [[2, 6, 6, 1],
         [2, 6, 6, 1]],

        [[5, 1, 1, 8],
         [8, 8, 0, 6]],

        [[5, 1, 1, 8],
         [4, 5, 2, 6]],

        [[5, 1, 1, 8],
         [8, 8, 4, 8]],

        [[5, 2, 8, 3],
         [8, 8, 0, 6]],

        [[5, 2, 8, 3],
         [4, 5, 2, 6]],

        [[5, 2, 8, 3],
         [2, 6, 6, 1]],

        [[3, 9, 2, 8],
         [8, 8, 0, 6]],

        [[3, 9, 2, 8],
         [4, 5, 2, 6]],

        [[3, 9, 2, 8],
         [8, 8, 4, 8]],

        [[0, 2, 0, 3],
         [8, 8, 0

In [101]:
index = torch.nonzero(encoding != 8)

In [102]:
encoding[index]

tensor([[[8, 8, 0, 6],
         [8, 8, 4, 8]],

        [[8, 8, 0, 6],
         [2, 6, 6, 1]],

        [[4, 5, 2, 6],
         [8, 8, 0, 6]],

        [[4, 5, 2, 6],
         [4, 5, 2, 6]],

        [[4, 5, 2, 6],
         [8, 8, 4, 8]],

        [[4, 5, 2, 6],
         [2, 6, 6, 1]],

        [[8, 8, 4, 8],
         [8, 8, 4, 8]],

        [[2, 6, 6, 1],
         [8, 8, 0, 6]],

        [[2, 6, 6, 1],
         [4, 5, 2, 6]],

        [[2, 6, 6, 1],
         [8, 8, 4, 8]],

        [[2, 6, 6, 1],
         [2, 6, 6, 1]],

        [[5, 1, 1, 8],
         [8, 8, 0, 6]],

        [[5, 1, 1, 8],
         [4, 5, 2, 6]],

        [[5, 1, 1, 8],
         [8, 8, 4, 8]],

        [[5, 2, 8, 3],
         [8, 8, 0, 6]],

        [[5, 2, 8, 3],
         [4, 5, 2, 6]],

        [[5, 2, 8, 3],
         [2, 6, 6, 1]],

        [[3, 9, 2, 8],
         [8, 8, 0, 6]],

        [[3, 9, 2, 8],
         [4, 5, 2, 6]],

        [[3, 9, 2, 8],
         [8, 8, 4, 8]],

        [[0, 2, 0, 3],
         [8, 8, 0

In [95]:
m_hat.shape

torch.Size([20, 10])

In [87]:
m_hat

tensor([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [-0., -0., -0., -0., -0., -0., -0., -0., -0., -0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [-0., -0., -0., -0., -0., -0., -0., -0., -0., -0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [-0., -0., -0., -0., -0., -0., -0., -0., -0., -0.],
        [-0., -0., -0., -0., -0., -0., -0., -0., -0., -0.],
        [-0., -0., -0., -0., -0., -0., -0., -0., -0., -0.],
        [-0., -0., -0., -0., -0., -0., -0., -0., -0., -0.],
        [-0., -0., -0., -0., -0., -0., -0., -0., -0., -0.],
        [-0., -0., -0., -0., -0., -0., -0., -0., -0., -0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
        [-0., -0., -0., -0., -0., -0., -0., -0., -0., -0.],
        [-0., -0., -0., -0., -0., -0., -0., -0., -0., -0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],


The next step is to implement:

$$ m_{\mu j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) \backslash \mu} \ \tanh^{-1} \left(  \hat{m}_{\nu j}  \right) + \beta(\rho_\xi)     \right)  \; \; ,  $$ 

In [42]:
for j in range(N):
    
    for mu in range(M):
        
        M_set = torch.where(encoding == j)[0]
        
        M_set_no_mu = M_set[torch.nonzero(M_set != mu).squeeze()]
        
        m[mu, j] = torch.tanh(torch.take(np.arctanh(m_hat[:, j]), M_set_no_mu).sum() + beta_prior)

In [43]:
m

tensor([[0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000],
        [0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000],
        [0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000],
        ...,
        [0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000],
        [0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000],
        [0.8000, 0.8000, 0.8000,  ..., 0.8000, 0.8000, 0.8000]])

The next step is to implement the pseudo-posterior, which is calculated after convergence of the messages above:

$$ m_{j} = \tanh \left(  \sum_{ \nu \in {\cal M} (j) } \tanh^{-1} \left(  \hat{m}_{\nu j}  \right)    + \beta(\rho_\xi)  \right)   \; \; ,  $$

In [44]:
m_bayes = []

for j in range(N):
    
    M_set = torch.where(encoding == j)[0]
       
    m_j = torch.tanh(torch.take(np.arctanh(m_hat[:, j]), M_set).sum() + beta_prior)
    
    m_bayes.append(m_j.item())
            
m_bayes

[0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,
 0.800000011920929,


In [45]:
len(m_bayes)

100

In [46]:
np.sign(m_bayes)

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])

Gathering all message iterations on functions:

In [167]:
def m_to_mhat(N, M, J_, m, beta, encoding):
    
    m_hat = torch.empty(M, N)
    
    for mu in range(M):
        for j in range(N):
                     
            # Keep only L(mu) which a are different of j
            index_no_j = torch.nonzero(encoding[mu] != j).squeeze()
            L_no_j = encoding[mu][index_no_j]
        
            # Message update        
            m_hat[mu, j] = torch.tanh( beta* J_[mu])*torch.take(m[mu], L_no_j).prod(dim=0)
            
    return m_hat


#####################################################################


def mhat_to_m(N, M, m_hat, beta_prior, encoding):
    
    m = torch.empty(M, N)
    
    for j in range(N):
        
        for mu in range(M):
            
            M_set = torch.where(encoding == j)[0]
        
            M_set_no_mu = M_set[torch.nonzero(M_set != mu).squeeze()]
            
            m[mu, j] = torch.tanh(torch.take(np.arctanh(m_hat[:, j]), M_set_no_mu).sum() + beta_prior)
            
    return m


#####################################################################


def m_bayes(N, m_hat_final, beta_prior, encoding):
    
    m_bayes = []

    for j in range(N):
           
        M_set = torch.where(encoding == j)[0]
  
        m_j = torch.tanh(torch.take(np.arctanh(m_hat_final[:, j]), M_set).sum() + beta_prior)
    
        m_bayes.append(m_j.item())
        
    return m_bayes

Now we construct a function for the iterative decoding.

In [168]:
def BP_LDPC(N, M, J_, beta, beta_prior, encoding, message, num_it= 100, verbose= 1):
     
    m_hat = torch.rand(M, N)
    m = torch.empty(M, N)
    
    for j in range(num_it):
        
        print('--it = %d' % j)
              
        m = mhat_to_m(N, M, m_hat, beta_prior, encoding)
        m_hat = m_to_mhat(N, M, J_, m, beta, encoding)
        
        ## Monitoring performace
        if (j % verbose) == 0:
            m_ = m_bayes(N, m_hat, beta_prior, encoding)
            print('overlap= ', torch.dot(message, torch.Tensor(np.sign(m_))).item() / N)
        
    m_final = m_bayes(N, m_hat, beta_prior, encoding)
    
    return np.sign(m_final)

In [49]:
opt_dec_Bayes = BP_LDPC(N, M, J_, beta, beta_prior, encoding, message[0], num_it= 20, verbose= 1)

--it = 0
overlap=  0.76
--it = 1
overlap=  0.76
--it = 2
overlap=  0.76
--it = 3
overlap=  0.76
--it = 4
overlap=  0.76
--it = 5
overlap=  0.76
--it = 6
overlap=  0.76
--it = 7
overlap=  0.76
--it = 8
overlap=  0.76
--it = 9
overlap=  0.76
--it = 10
overlap=  0.76
--it = 11
overlap=  0.76
--it = 12
overlap=  0.76
--it = 13
overlap=  0.76
--it = 14
overlap=  0.76
--it = 15
overlap=  0.76
--it = 16
overlap=  0.76
--it = 17
overlap=  0.76
--it = 18
overlap=  0.76
--it = 19
overlap=  0.76


Ok. It is still slow (we will worrie about this latter), but it appears to work.

Observe that we arbitrarily consider a certain `num_it`. Naturally, a important question remains: how do we monitor the convergence of BP?

## Quick test on previous codes and messages

In [157]:
# Message lengh
N = 250

# Codeword lengh
M = 500

# Non-zero elements per row of the generation matrix
K = 4

# Number of messages
n = 100

# Noisy channel
p = 0.3
beta = 0.5*np.log( (1 - p) / p)

# Message prior
p_prior = 0.01
beta_prior = 0.5*np.log( (1 - p_prior) / p_prior)

In [136]:
encoding = torch.load('codes/code_N_250_M_500_K_4_p_03_p_prior_001_j_0.pt')
message = torch.load('codes/message_N_250_M_500_K_4_p_03_p_prior_001.pt')

In [137]:
encoding

tensor([[ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1., -1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        ...,
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.]])

In [145]:
print(message)
message.shape

tensor([[ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1., -1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        ...,
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.],
        [ 1.,  1.,  1.,  ...,  1.,  1.,  1.]])


torch.Size([100, 250])

Well, we made a mistake saving the codes. We have actually saved the messages. That is a problem. For now, let us generate a new code.

In [139]:
encoding  = torch.randint(0, N, [M, K])
encoding

tensor([[ 94,  21, 204, 242],
        [162,  49, 100, 151],
        [230, 198, 123,  57],
        ...,
        [ 87, 176,   1, 211],
        [ 99,  72,  51,  91],
        [248, 145, 105, 154]])

Encoding.

In [140]:
# Initializing
J0 = torch.take(message[0], encoding).prod(dim=1)
J0 = J0.unsqueeze(0)

for j in range(1, message.shape[0]):
    
    J0_ = torch.take(message[j], encoding).prod(dim=1)
    J0_ = J0_.unsqueeze(0)
    
    J0 = torch.cat((J0, J0_), dim= 0)

In [144]:
print(J0)
J0.shape

tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]])


torch.Size([100, 500])

Corrupted version.

In [142]:
J = J0.clone()

random = torch.rand(J.shape)
                      
for j in range(J.shape[0]):
    for k in range(J.shape[1]):
          
        if random[j, k] <= p:
            J[j, k] = -J[j, k]

In [143]:
print(J)
J.shape

tensor([[-1., -1.,  1.,  ...,  1., -1.,  1.],
        [-1.,  1., -1.,  ...,  1.,  1.,  1.],
        [ 1., -1., -1.,  ..., -1.,  1.,  1.],
        ...,
        [-1.,  1.,  1.,  ..., -1.,  1.,  1.],
        [ 1.,  1., -1.,  ...,  1.,  1., -1.],
        [ 1.,  1.,  1.,  ...,  1., -1.,  1.]])


torch.Size([100, 500])

In [149]:
J[0].shape

torch.Size([500])

In [151]:
message[0].shape

torch.Size([250])

In [155]:
print(N)
print(M)
print(encoding.shape)
print(J.shape)
print(message.shape)

250
500
torch.Size([500, 4])
torch.Size([100, 500])
torch.Size([100, 250])


In [ ]:
overlap_list = []

for k in range(int(J.shape[0] / 10 )):
    
    print('----message %d' % k)
       
    opt_dec_Bayes = BP_LDPC(N, M, J[k], beta, beta_prior, encoding, message[k], num_it= 10, verbose= 1)
    
    overlap_list.append(opt_dec_Bayes)

In [127]:
list_ = []

In [121]:
for j in range(10):
    list_.append([2, j])

In [126]:
l = [0, 2, 4, 5, 3, 5]
p = [98.8, 89, 5.9, 3.4]

In [128]:
list_.append(l)

In [129]:
list_

[[0, 2, 4, 5, 3, 5]]

In [131]:
list_.append(p)

In [132]:
list_

[[0, 2, 4, 5, 3, 5], [98.8, 89, 5.9, 3.4]]

In [133]:
list_[0]

[0, 2, 4, 5, 3, 5]